# Importing Necessary Libraries

In [1]:
import random
import copy
import json, os
from collections import Counter
from PIL import Image
from tqdm.notebook import tqdm, trange

import clip
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.utils.data import DataLoader
from torchvision import transforms
from datasets import Dataset, load_dataset, load_from_disk, concatenate_datasets
from sklearn.metrics import classification_report, top_k_accuracy_score

from transformers import pipeline
from transformers import CLIPProcessor, CLIPModel, CLIPImageProcessor, CLIPTokenizer
from prompt import le_prompt, prompt, negative_prompt, description

import warnings
warnings.filterwarnings("ignore")

random.seed(24)

# Generate Language Detailed Prompts

In [ ]:
messages = [
    {"role": "system", "content": "You are a professional botanist with speciality in describing and picturising rare plants in great detail"},
    {"role": "user", "content": """
    Rephrase the following description of {} plant. Remove the camera details and 
    maintain the features of the plant. Add more detailed features if required to 
    describe the flower. Just provide the rephrased description of the plant in 1 
    paragraph.

    Description:
    {}"""},
]

pipe = pipeline("text-generation", model="TinyLlama/TinyLlama-1.1B-Chat-v1.0", torch_dtype=torch.bfloat16, device_map="auto")

tiny_prompts = []

for flower in prompt:
    for i in range(320):
        tiny_prompt = copy.deepcopy(messages)
        tiny_prompt[1]['content'] = tiny_prompt[1]['content'].format(flower, prompt[flower])
        tiny_prompts.append(tiny_prompt)

outputs = pipe(tiny_prompts, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)

# Generate the label mapping to indexes and descriptions

In [2]:
# Label Maps
l2i = {}
i2l = []

# Descriptions
desc = []

for i, plant in enumerate(prompt):
    l2i[plant] = i
    i2l.append(plant)
    desc.append(description[plant])

print(i2l)

['Rufflesia-Arnoldii', 'Encephalartos-Woodii', 'Amorphophallus-Titanum', 'Ghost-Orchid', 'Dracaena-Cinnabari']


# Load the dataset

In [40]:
ds = load_dataset("imagefolder", data_dir="./data/images")
syn = load_dataset("imagefolder", data_dir="./data/synthetic/")
fsl = load_dataset("imagefolder", data_dir="./data/synthetic-few-shot/")

Resolving data files:   0%|          | 0/25 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/250 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/1600 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/1000 [00:00<?, ?it/s]

# Apply processing over the data

In [41]:
image_size = (224, 224)

ds = ds.map(lambda batch: {'pixel_values': transforms.ToTensor()(batch['image'].resize(image_size))}, batch_size=32)
syn = syn.map(lambda batch: {'pixel_values': transforms.ToTensor()(batch['image'].resize(image_size))}, batch_size=32)
fsl = fsl.map(lambda batch: {'pixel_values': transforms.ToTensor()(batch['image'].resize(image_size))}, batch_size=32)

ds = ds.remove_columns(['image'])
syn = syn.remove_columns(['image'])
fsl = fsl.remove_columns(['image'])

ds = ds.align_labels_with_mapping(l2i, "label")
syn = syn.align_labels_with_mapping(l2i, "label")
fsl = fsl.align_labels_with_mapping(l2i, "label")

ds = ds.with_format('torch')
syn = syn.with_format('torch')
fsl = fsl.with_format('torch')

# Load the CLIP model and utilities

In [5]:
model_name = 'openai/clip-vit-large-patch14'

# model = CLIPModel.from_pretrained(model_name, device_map='auto')
processor = CLIPProcessor.from_pretrained(model_name)

# Utility for evaluating the model

In [6]:
def eval(eval_model, cls):
    eval_model.eval()
    
    test_labels = []
    test_probs = []
    
    test_loader = torch.utils.data.DataLoader(ds['test'], batch_size=32)
    for batch in tqdm(test_loader, total=len(test_loader)):
        with torch.no_grad():
            inputs = processor(text=cls, images=batch['pixel_values'], return_tensors="pt", padding=True)
            inputs = {k: v.to('cuda') for k, v in inputs.items()}
            
            outputs = eval_model(**inputs)
            logits_per_image = outputs.logits_per_image
            probs = logits_per_image.softmax(dim=1)
        
            del inputs
            torch.cuda.empty_cache()
        
            test_labels.append(batch['label'].cpu())
            test_probs.append(probs.cpu())
    
    test_labels = torch.cat(test_labels)
    test_probs = torch.cat(test_probs)

    return test_labels, test_probs

## Evaluate accuray with name only labels

In [7]:
y_true, y_pred = eval(model, [f'This is image of the {plant} plant' for plant in i2l])
print(classification_report(y_true, y_pred.argmax(dim=1, keepdim=True)))

  0%|          | 0/8 [00:00<?, ?it/s]

It looks like you are trying to rescale already rescaled images. If the input images have pixel values between 0 and 1, set `do_rescale=False` to avoid rescaling them again.


              precision    recall  f1-score   support

           0       0.18      0.54      0.27        50
           1       0.27      0.08      0.12        50
           2       0.27      0.32      0.29        50
           3       0.58      0.28      0.38        50
           4       0.00      0.00      0.00        50

    accuracy                           0.24       250
   macro avg       0.26      0.24      0.21       250
weighted avg       0.26      0.24      0.21       250



## Evaluate accuray with detailed labels

In [8]:
y_true, y_pred = eval(model, desc)
print(classification_report(y_true, y_pred.argmax(dim=1, keepdim=True)))

  0%|          | 0/8 [00:00<?, ?it/s]

              precision    recall  f1-score   support

           0       0.20      1.00      0.34        50
           1       0.00      0.00      0.00        50
           2       0.00      0.00      0.00        50
           3       1.00      0.04      0.08        50
           4       0.00      0.00      0.00        50

    accuracy                           0.21       250
   macro avg       0.24      0.21      0.08       250
weighted avg       0.24      0.21      0.08       250



# Train CLIP Model with synthetic data

In [7]:
def compute_loss_and_accuracy(outputs, labels):
    logits = outputs.logits_per_image  # Shape: (batch_size, num_labels)
    labels = labels.to(logits.device)
    
    # Compute the cross-entropy loss
    loss = F.cross_entropy(logits, labels)
    
    # Compute the accuracy
    _, predicted_labels = torch.max(logits, dim=1)
    accuracy = torch.sum(predicted_labels == labels) / labels.size(0)
    
    return loss, accuracy

In [8]:
def train_clip_model(
    model, 
    processor, 
    train_dataset, 
    prompts, 
    num_epochs=10, 
    batch_size=32, 
    learning_rate=5e-6, 
    device='cuda', 
    warmup_steps=100, 
    accumulation_steps=4, 
    max_grad_norm=1.0
):
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=0.01)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)
    model.to(device)

    progress_bar = tqdm(total=num_epochs * len(train_dataloader), unit='batch')
    global_step = 0
    
    for epoch in range(num_epochs):
        model.train()
        epoch_loss = 0.0
        epoch_accuracy = 0.0
        num_batch = len(train_dataloader)

        for step, batch in enumerate(train_dataloader):
            if isinstance(prompts, dict):
                cls = [random.sample(prompts[plant], k=1)[0] for plant in i2l]
            else:
                cls = prompts
            
            inputs = processor(
                images=batch["pixel_values"], 
                text=cls, 
                return_tensors="pt", 
                padding=True,
                max_length=77,
                truncation=True,
            )
            inputs = {k: v.to(device) for k, v in inputs.items()}

            outputs = model(**inputs)
            loss, accuracy = compute_loss_and_accuracy(outputs, batch['label'])

            loss = loss / accumulation_steps
            loss.backward()

            if (step + 1) % accumulation_steps == 0:
                # Gradient Clipping
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=max_grad_norm)
                
                optimizer.step()
                optimizer.zero_grad()

            epoch_loss += loss.item()
            epoch_accuracy += accuracy.item()

            progress_bar.set_postfix(epoch=(epoch+1)+(step/num_batch), loss=loss.item(), accuracy=accuracy.item())
            progress_bar.update(1)

            global_step += 1

            # Warmup Steps
            if global_step < warmup_steps:
                warmup_percent = global_step / warmup_steps
                curr_lr = learning_rate * warmup_percent
                optimizer.param_groups[0]['lr'] = curr_lr

        scheduler.step()

        epoch_loss /= len(train_dataloader)
        epoch_accuracy /= len(train_dataloader)

        # Print epoch summary
        print(f"Epoch {epoch+1}/{num_epochs} - Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}")

    progress_bar.close()
    return model

def freeze_layers(model, num_frozen_layers):
    for i, param in enumerate(model.parameters()):
        if i < num_frozen_layers:
            param.requires_grad = False
        else:
            param.requires_grad = True

### Freeze Layers for stable learning convergence

In [39]:
# Load the pre-trained CLIP model
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")

# Freeze the first few layers of the model
num_frozen_layers = 100
freeze_layers(model, num_frozen_layers)

In [12]:
num_epochs = 5
batch_size = 8
learning_rate = 1e-5
device = 'cuda'

# Train the model
gen_prompts = json.load(open('./generated_text.json', 'r'))
fine_tuned_model = train_clip_model(model, processor, syn['train'], gen_prompts, num_epochs, batch_size, learning_rate, device)

# Save the fine-tuned model
fine_tuned_model.save_pretrained("./models/zsl-desc-tuned")

  0%|          | 0/1000 [00:00<?, ?batch/s]

Epoch 1/5 - Loss: 0.3641, Accuracy: 0.4950
Epoch 2/5 - Loss: 0.0536, Accuracy: 0.9231
Epoch 3/5 - Loss: 0.0361, Accuracy: 0.9513
Epoch 4/5 - Loss: 0.0265, Accuracy: 0.9688
Epoch 5/5 - Loss: 0.0025, Accuracy: 0.9988


## Evaluate accuray with name only labels

In [16]:
y_true, y_pred = eval(fine_tuned_model, i2l)
print(classification_report(y_true, y_pred.argmax(dim=1, keepdim=True)))

  0%|          | 0/8 [00:00<?, ?it/s]

              precision    recall  f1-score   support

           0       0.89      0.50      0.64        50
           1       1.00      0.16      0.28        50
           2       0.35      0.24      0.29        50
           3       0.34      0.98      0.51        50
           4       0.83      0.60      0.70        50

    accuracy                           0.50       250
   macro avg       0.68      0.50      0.48       250
weighted avg       0.68      0.50      0.48       250



## Evaluate accuray with detailed description of labels

In [15]:
y_true, y_pred = eval(fine_tuned_model, desc)
print(classification_report(y_true, y_pred.argmax(dim=1, keepdim=True)))

  0%|          | 0/8 [00:00<?, ?it/s]

              precision    recall  f1-score   support

           0       0.91      0.58      0.71        50
           1       0.85      0.34      0.49        50
           2       0.50      0.24      0.32        50
           3       0.39      0.98      0.56        50
           4       0.73      0.72      0.73        50

    accuracy                           0.57       250
   macro avg       0.68      0.57      0.56       250
weighted avg       0.68      0.57      0.56       250



In [17]:
model = CLIPModel.from_pretrained("./models/zsl-desc-tuned/")

# Few shot Learning with CLIP

In [5]:
def minmax_scaler(tensor):
    b, n, h, w = tensor.size()
    min_val = tensor.view(b, n, -1).min(dim=-1, keepdims=True).values.view([b, n, 1, 1])
    max_val = tensor.view(b, n, -1).max(dim=-1, keepdims=True).values.view([b, n, 1, 1])
    scaled_tensor = (tensor - min_val) / (max_val - min_val + 1e-6)
    return scaled_tensor

transform = transforms.Compose([
    transforms.RandomResizedCrop(size=224),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomGrayscale(p=0.1),
    transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5)),
    transforms.Lambda(minmax_scaler)  # Apply channel-wise min-max scaling
])

In [6]:
def transform_sample(sample):
    pixel_values = sample["pixel_values"]
    transformed_pixel_values = transform(pixel_values)
    sample["pixel_values"] = transformed_pixel_values
    return sample

def create_extended_dataset(dataset, num_times=2):
    extended_datasets = []
    for _ in range(num_times - 1):
        transformed_dataset = dataset.map(transform_sample, batched=True, batch_size=4)
        extended_datasets.append(transformed_dataset)  
    extended_datasets.append(dataset)
    
    extended_dataset = concatenate_datasets(extended_datasets)
    return extended_dataset

## Real Image Only Learning

In [11]:
num_times = 10
train_dataset = create_extended_dataset(ds["train"], num_times)

Map:   0%|          | 0/25 [00:00<?, ? examples/s]

In [14]:
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32", device_map='auto')

num_frozen_layers = 100
freeze_layers(model, num_frozen_layers)

num_epochs = 5
batch_size = 8
learning_rate = 1e-5
device = 'cuda'

gen_prompts = json.load(open('./generated_text.json', 'r'))
fine_tuned_model = train_clip_model(model, processor, train_dataset, gen_prompts, num_epochs, batch_size, learning_rate, device)

fine_tuned_model.save_pretrained("./models/real-desc-tuned")

  0%|          | 0/160 [00:00<?, ?batch/s]

Epoch 1/5 - Loss: 0.6578, Accuracy: 0.1914
Epoch 2/5 - Loss: 0.4673, Accuracy: 0.2461
Epoch 3/5 - Loss: 0.4803, Accuracy: 0.2109
Epoch 4/5 - Loss: 0.4273, Accuracy: 0.2773
Epoch 5/5 - Loss: 0.3754, Accuracy: 0.3672


In [15]:
y_true, y_pred = eval(fine_tuned_model, i2l)
print(classification_report(y_true, y_pred.argmax(dim=1, keepdim=True)))

y_true, y_pred = eval(fine_tuned_model, desc)
print(classification_report(y_true, y_pred.argmax(dim=1, keepdim=True)))

  0%|          | 0/8 [00:00<?, ?it/s]

              precision    recall  f1-score   support

           0       0.20      1.00      0.33        50
           1       0.00      0.00      0.00        50
           2       0.00      0.00      0.00        50
           3       0.00      0.00      0.00        50
           4       0.00      0.00      0.00        50

    accuracy                           0.20       250
   macro avg       0.04      0.20      0.07       250
weighted avg       0.04      0.20      0.07       250



  0%|          | 0/8 [00:00<?, ?it/s]

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        50
           1       0.22      1.00      0.37        50
           2       0.00      0.00      0.00        50
           3       1.00      0.08      0.15        50
           4       0.87      0.40      0.55        50

    accuracy                           0.30       250
   macro avg       0.42      0.30      0.21       250
weighted avg       0.42      0.30      0.21       250



## Phase Wise (Real -> Synthetic)

In [21]:
def get_few_shot_train_data(n_shot=1):
    train_dataset = fsl['train']
    unique_labels = train_dataset.unique('label')
    new_dataset = []
    
    for label in unique_labels:
        filtered_dataset = train_dataset.filter(
            lambda example: example['label'] == label, 
            num_proc=1,
        ).select(range(40 * n_shot))
        new_dataset.append(filtered_dataset)
    
    new_fsl = concatenate_datasets(new_dataset)
    return new_fsl

## 1-shot training results

In [24]:
num_epochs = 5
batch_size = 8
learning_rate = 1e-5
device = 'cuda'

model = CLIPModel.from_pretrained("./models/real-desc-tuned")
model.train()

num_frozen_layers = 100
freeze_layers(model, num_frozen_layers)

train_dataset = get_few_shot_train_data(n_shot=1)

gen_prompts = json.load(open('./generated_text.json', 'r'))
fine_tuned_model = train_clip_model(model, processor, train_dataset, gen_prompts, num_epochs, batch_size, learning_rate, device)

fine_tuned_model.save_pretrained("./models/real-syn-1-shot")

  0%|          | 0/125 [00:00<?, ?batch/s]

Epoch 1/5 - Loss: 0.3587, Accuracy: 0.3850
Epoch 2/5 - Loss: 0.3115, Accuracy: 0.4500
Epoch 3/5 - Loss: 0.2633, Accuracy: 0.5600
Epoch 4/5 - Loss: 0.2986, Accuracy: 0.5000
Epoch 5/5 - Loss: 0.2191, Accuracy: 0.6250


In [25]:
y_true, y_pred = eval(fine_tuned_model, i2l)
print(classification_report(y_true, y_pred.argmax(dim=1, keepdim=True)))

y_true, y_pred = eval(fine_tuned_model, desc)
print(classification_report(y_true, y_pred.argmax(dim=1, keepdim=True)))

  0%|          | 0/8 [00:00<?, ?it/s]

              precision    recall  f1-score   support

           0       0.26      1.00      0.42        50
           1       0.00      0.00      0.00        50
           2       1.00      0.08      0.15        50
           3       0.79      0.46      0.58        50
           4       0.85      0.46      0.60        50

    accuracy                           0.40       250
   macro avg       0.58      0.40      0.35       250
weighted avg       0.58      0.40      0.35       250



  0%|          | 0/8 [00:00<?, ?it/s]

              precision    recall  f1-score   support

           0       0.91      0.20      0.33        50
           1       0.33      0.44      0.38        50
           2       0.49      0.34      0.40        50
           3       0.48      0.90      0.63        50
           4       0.80      0.70      0.74        50

    accuracy                           0.52       250
   macro avg       0.60      0.52      0.50       250
weighted avg       0.60      0.52      0.50       250



## 2-shot training results

In [26]:
num_epochs = 5
batch_size = 8
learning_rate = 1e-5
device = 'cuda'

model = CLIPModel.from_pretrained("./models/real-desc-tuned")
model.train()

num_frozen_layers = 100
freeze_layers(model, num_frozen_layers)

train_dataset = get_few_shot_train_data(n_shot=2)

gen_prompts = json.load(open('./generated_text.json', 'r'))
fine_tuned_model = train_clip_model(model, processor, train_dataset, gen_prompts, num_epochs, batch_size, learning_rate, device)

fine_tuned_model.save_pretrained("./models/real-syn-2-shot")

  0%|          | 0/250 [00:00<?, ?batch/s]

Epoch 1/5 - Loss: 0.3411, Accuracy: 0.4425
Epoch 2/5 - Loss: 0.2697, Accuracy: 0.5250
Epoch 3/5 - Loss: 0.2104, Accuracy: 0.6500
Epoch 4/5 - Loss: 0.1715, Accuracy: 0.7225
Epoch 5/5 - Loss: 0.1099, Accuracy: 0.8375


In [27]:
y_true, y_pred = eval(fine_tuned_model, i2l)
print(classification_report(y_true, y_pred.argmax(dim=1, keepdim=True)))

y_true, y_pred = eval(fine_tuned_model, desc)
print(classification_report(y_true, y_pred.argmax(dim=1, keepdim=True)))

  0%|          | 0/8 [00:00<?, ?it/s]

              precision    recall  f1-score   support

           0       0.27      0.98      0.42        50
           1       0.00      0.00      0.00        50
           2       1.00      0.06      0.11        50
           3       0.88      0.46      0.61        50
           4       0.82      0.64      0.72        50

    accuracy                           0.43       250
   macro avg       0.59      0.43      0.37       250
weighted avg       0.59      0.43      0.37       250



  0%|          | 0/8 [00:00<?, ?it/s]

              precision    recall  f1-score   support

           0       0.55      0.94      0.69        50
           1       0.62      0.36      0.46        50
           2       0.57      0.26      0.36        50
           3       0.66      0.86      0.75        50
           4       0.81      0.76      0.78        50

    accuracy                           0.64       250
   macro avg       0.64      0.64      0.61       250
weighted avg       0.64      0.64      0.61       250



## 3-shot training results

In [31]:
num_epochs = 5
batch_size = 8
learning_rate = 1e-5
device = 'cuda'

model = CLIPModel.from_pretrained("./models/real-desc-tuned")
model.train()

num_frozen_layers = 100
freeze_layers(model, num_frozen_layers)

train_dataset = get_few_shot_train_data(n_shot=3)

gen_prompts = json.load(open('./generated_text.json', 'r'))
fine_tuned_model = train_clip_model(model, processor, train_dataset, gen_prompts, num_epochs, batch_size, learning_rate, device)

fine_tuned_model.save_pretrained("./models/real-syn-3-shot")

  0%|          | 0/375 [00:00<?, ?batch/s]

Epoch 1/5 - Loss: 0.3403, Accuracy: 0.4283
Epoch 2/5 - Loss: 0.2672, Accuracy: 0.5567
Epoch 3/5 - Loss: 0.2571, Accuracy: 0.6050
Epoch 4/5 - Loss: 0.1776, Accuracy: 0.7017
Epoch 5/5 - Loss: 0.1111, Accuracy: 0.8383


In [32]:
y_true, y_pred = eval(fine_tuned_model, i2l)
print(classification_report(y_true, y_pred.argmax(dim=1, keepdim=True)))

y_true, y_pred = eval(fine_tuned_model, desc)
print(classification_report(y_true, y_pred.argmax(dim=1, keepdim=True)))

  0%|          | 0/8 [00:00<?, ?it/s]

              precision    recall  f1-score   support

           0       0.29      0.86      0.43        50
           1       0.50      0.04      0.07        50
           2       0.57      0.32      0.41        50
           3       0.76      0.64      0.70        50
           4       0.93      0.52      0.67        50

    accuracy                           0.48       250
   macro avg       0.61      0.48      0.46       250
weighted avg       0.61      0.48      0.46       250



  0%|          | 0/8 [00:00<?, ?it/s]

              precision    recall  f1-score   support

           0       0.66      0.62      0.64        50
           1       0.50      0.30      0.38        50
           2       0.47      0.68      0.56        50
           3       0.77      0.82      0.80        50
           4       0.85      0.82      0.84        50

    accuracy                           0.65       250
   macro avg       0.65      0.65      0.64       250
weighted avg       0.65      0.65      0.64       250



## 4-shot training results

In [33]:
num_epochs = 5
batch_size = 8
learning_rate = 1e-5
device = 'cuda'

model = CLIPModel.from_pretrained("./models/real-desc-tuned")
model.train()

num_frozen_layers = 100
freeze_layers(model, num_frozen_layers)

train_dataset = get_few_shot_train_data(n_shot=4)

gen_prompts = json.load(open('./generated_text.json', 'r'))
fine_tuned_model = train_clip_model(model, processor, train_dataset, gen_prompts, num_epochs, batch_size, learning_rate, device)

fine_tuned_model.save_pretrained("./models/real-syn-4-shot")

  0%|          | 0/500 [00:00<?, ?batch/s]

Epoch 1/5 - Loss: 0.3353, Accuracy: 0.4387
Epoch 2/5 - Loss: 0.3269, Accuracy: 0.4475
Epoch 3/5 - Loss: 0.2392, Accuracy: 0.6000
Epoch 4/5 - Loss: 0.1593, Accuracy: 0.7275
Epoch 5/5 - Loss: 0.1022, Accuracy: 0.8450


In [34]:
y_true, y_pred = eval(fine_tuned_model, i2l)
print(classification_report(y_true, y_pred.argmax(dim=1, keepdim=True)))

y_true, y_pred = eval(fine_tuned_model, desc)
print(classification_report(y_true, y_pred.argmax(dim=1, keepdim=True)))

  0%|          | 0/8 [00:00<?, ?it/s]

              precision    recall  f1-score   support

           0       0.29      0.94      0.44        50
           1       0.00      0.00      0.00        50
           2       0.75      0.24      0.36        50
           3       0.78      0.62      0.69        50
           4       1.00      0.60      0.75        50

    accuracy                           0.48       250
   macro avg       0.56      0.48      0.45       250
weighted avg       0.56      0.48      0.45       250



  0%|          | 0/8 [00:00<?, ?it/s]

              precision    recall  f1-score   support

           0       0.73      0.82      0.77        50
           1       0.67      0.20      0.31        50
           2       0.47      0.76      0.58        50
           3       0.82      0.80      0.81        50
           4       0.88      0.86      0.87        50

    accuracy                           0.69       250
   macro avg       0.71      0.69      0.67       250
weighted avg       0.71      0.69      0.67       250



## 5-shot training results

In [38]:
num_epochs = 5
batch_size = 8
learning_rate = 1e-5
device = 'cuda'

model = CLIPModel.from_pretrained("./models/real-desc-tuned")
model.train()

num_frozen_layers = 100
freeze_layers(model, num_frozen_layers)

train_dataset = get_few_shot_train_data(n_shot=5)

gen_prompts = json.load(open('./generated_text.json', 'r'))
fine_tuned_model = train_clip_model(model, processor, train_dataset, gen_prompts, num_epochs, batch_size, learning_rate, device)

fine_tuned_model.save_pretrained("./models/real-syn-5-shot")

  0%|          | 0/625 [00:00<?, ?batch/s]

Epoch 1/5 - Loss: 0.1133, Accuracy: 0.8170
Epoch 2/5 - Loss: 0.1557, Accuracy: 0.7740
Epoch 3/5 - Loss: 0.1096, Accuracy: 0.8300
Epoch 4/5 - Loss: 0.0533, Accuracy: 0.9240
Epoch 5/5 - Loss: 0.0408, Accuracy: 0.9410


In [39]:
y_true, y_pred = eval(fine_tuned_model, i2l)
print(classification_report(y_true, y_pred.argmax(dim=1, keepdim=True)))

y_true, y_pred = eval(fine_tuned_model, desc)
print(classification_report(y_true, y_pred.argmax(dim=1, keepdim=True)))

  0%|          | 0/8 [00:00<?, ?it/s]

              precision    recall  f1-score   support

           0       0.77      0.80      0.78        50
           1       0.50      0.10      0.17        50
           2       0.44      0.86      0.59        50
           3       0.86      0.74      0.80        50
           4       0.85      0.82      0.84        50

    accuracy                           0.66       250
   macro avg       0.69      0.66      0.63       250
weighted avg       0.69      0.66      0.63       250



  0%|          | 0/8 [00:00<?, ?it/s]

              precision    recall  f1-score   support

           0       0.77      0.94      0.85        50
           1       0.70      0.28      0.40        50
           2       0.50      0.84      0.63        50
           3       0.97      0.70      0.81        50
           4       0.84      0.82      0.83        50

    accuracy                           0.72       250
   macro avg       0.76      0.72      0.70       250
weighted avg       0.76      0.72      0.70       250



# Multimodal Models

In [7]:
import re

from llava.conversation import conv_templates, SeparatorStyle
from llava.mm_utils import (
    process_images,
    tokenizer_image_token,
    get_model_name_from_path,
)
from llava.constants import (
    IMAGE_TOKEN_INDEX,
    DEFAULT_IMAGE_TOKEN,
    DEFAULT_IM_START_TOKEN,
    DEFAULT_IM_END_TOKEN,
    IMAGE_PLACEHOLDER,
)

from llava.model.builder import load_pretrained_model

In [9]:
model_path = "liuhaotian/llava-v1.6-vicuna-7b"

tokenizer, model, image_processor, context_len = load_pretrained_model(
    model_path=model_path,
    model_base=None,
    model_name=get_model_name_from_path(model_path),
    load_4bit=True
)

You are using a model of type llava to instantiate a model of type llava_llama. This is not supported for all configurations of models and can yield errors.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
def eval_model(model, model_name, query, images):
    qs = query
    image_token_se = DEFAULT_IM_START_TOKEN + DEFAULT_IMAGE_TOKEN + DEFAULT_IM_END_TOKEN
    if IMAGE_PLACEHOLDER in qs:
        if model.config.mm_use_im_start_end:
            qs = re.sub(IMAGE_PLACEHOLDER, image_token_se, qs)
        else:
            qs = re.sub(IMAGE_PLACEHOLDER, DEFAULT_IMAGE_TOKEN, qs)
    else:
        if model.config.mm_use_im_start_end:
            qs = image_token_se + "\n" + qs
        else:
            qs = DEFAULT_IMAGE_TOKEN + "\n" + qs

    if "llama-2" in model_name.lower():
        conv_mode = "llava_llama_2"
    elif "mistral" in model_name.lower():
        conv_mode = "mistral_instruct"
    elif "v1.6-34b" in model_name.lower():
        conv_mode = "chatml_direct"
    elif "v1" in model_name.lower():
        conv_mode = "llava_v1"
    elif "mpt" in model_name.lower():
        conv_mode = "mpt"
    else:
        conv_mode = "llava_v0"

    conv = conv_templates[conv_mode].copy()
    conv.append_message(conv.roles[0], qs)
    conv.append_message(conv.roles[1], None)
    prompt = conv.get_prompt()

    image_sizes = [x.size for x in images]
    images_tensor = process_images(
        images,
        image_processor,
        model.config
    ).to(model.device, dtype=torch.float16)

    input_ids = (
        tokenizer_image_token(prompt, tokenizer, IMAGE_TOKEN_INDEX, return_tensors="pt")
        .unsqueeze(0)
        .cuda()
    )

    with torch.inference_mode():
        output_ids = model.generate(
            input_ids,
            images=images_tensor,
            image_sizes=image_sizes,
            do_sample=False,
            temperature=0,
            top_p=None,
            num_beams=1,
            max_new_tokens=512,
            use_cache=True,
        )

    outputs = tokenizer.batch_decode(output_ids, skip_special_tokens=True)[0].strip()
    return outputs

## Zero-shot evaluation on LLaVa (MLLM)

In [15]:
query = f"""
{IMAGE_PLACEHOLDER} 

Classify this image in one of the following classes
- Rufflesia-Arnoldii
- Encephalartos-Woodii
- Amorphophallus-Titanum
- Ghost-Orchid
- Dracaena-Cinnabari

Only give the class name for the class which you are 
highly confident the image belongs to.
"""

y_true = []
y_pred = []

for plant in tqdm(prompt):
    for i in trange(50):
        pred = eval_model(
            model=model, 
            model_name=get_model_name_from_path(model_path),
            query=query,
            images=[Image.open(f'./data/test/{plant}/{i+1:02d}.jpeg')]
        )
        
        y_true.append(plant)
        y_pred.append(pred)

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

In [16]:
print(classification_report(y_true, y_pred))

                        precision    recall  f1-score   support

Amorphophallus-Titanum       1.00      0.06      0.11        50
    Dracaena-Cinnabari       0.00      0.00      0.00        50
  Encephalartos-Woodii       0.18      0.20      0.19        50
          Ghost-Orchid       1.00      0.22      0.36        50
    Rufflesia-Arnoldii       0.28      1.00      0.43        50

              accuracy                           0.30       250
             macro avg       0.49      0.30      0.22       250
          weighted avg       0.49      0.30      0.22       250



## Evaluation after Fine-Tuning

### Fine-tuned zero shot evaluation

In [11]:
lora_path = '/home/LLaVA/checkpoints/llava-vicuna-v1-6-7b-finetune_qlora_plant/'
model.load_adapter(lora_path)

In [13]:
query = f"""
{IMAGE_PLACEHOLDER} 

Classify this image in one of the following classes
- Rufflesia-Arnoldii
- Encephalartos-Woodii
- Amorphophallus-Titanum
- Ghost-Orchid
- Dracaena-Cinnabari

Only give the class name for the class which you are 
highly confident the image belongs to.
"""

y_true = []
y_pred = []

for plant in tqdm(prompt):
    for i in trange(50):
        pred = eval_model(
            model=model, 
            model_name=get_model_name_from_path(model_path),
            query=query,
            images=[Image.open(f'./data/test/{plant}/{i+1:02d}.jpeg')]
        )
        
        y_true.append(plant)
        y_pred.append(pred)

print(classification_report(y_true, y_pred))

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

                        precision    recall  f1-score   support

Amorphophallus-Titanum       0.30      1.00      0.46        50
    Dracaena-Cinnabari       0.33      0.02      0.04        50
  Encephalartos-Woodii       0.20      0.04      0.07        50
          Ghost-Orchid       0.94      1.00      0.97        50
    Rufflesia-Arnoldii       0.00      0.00      0.00        50

              accuracy                           0.41       250
             macro avg       0.35      0.41      0.31       250
          weighted avg       0.35      0.41      0.31       250



# ViT Classifier

In [42]:
from datasets import load_dataset
from transformers import ViTImageProcessor, ViTForImageClassification, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, f1_score

# Initialize the image processor and model
image_processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224', do_rescale=False)
model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224', num_labels=5, ignore_mismatched_sizes=True)

# Define the data preprocessing function
def preprocess_function(examples):
    inputs = image_processor(examples['pixel_values'], return_tensors="pt")
    inputs['labels'] = examples['label']
    return inputs

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([5]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([5, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Real Data Only

In [43]:
num_times = 20
train_ds = create_extended_dataset(ds['train'], num_times)
processed_dataset = train_ds.map(preprocess_function, batched=True)

# train_ds = train_dataset.train_test_split(
#     test_size=0.2, 
#     shuffle=True, 
#     seed=42, 
#     stratify_by_column='label'
# )

In [45]:
# Define the training arguments
training_args = TrainingArguments(
    output_dir='./models',
    num_train_epochs=5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    learning_rate=2e-6,
    warmup_steps=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model='f1',
    logging_dir='./logs',
    logging_steps=1,
    push_to_hub=False,
)

# Define the compute metrics function
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='weighted')
    return {'accuracy': acc, 'f1': f1}

# Create the Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=processed_dataset,
    eval_dataset=ds['test'],
    compute_metrics=compute_metrics,
)

# Train the model
trainer.train()

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,1.384600,1.554926,0.364000,0.386581
2,0.821600,1.419253,0.424000,0.446194
3,0.477800,1.328774,0.476000,0.491890
4,0.389000,1.274997,0.504000,0.517480
5,0.486300,1.258621,0.516000,0.527586


TrainOutput(global_step=160, training_loss=0.771074716746807, metrics={'train_runtime': 100.0261, 'train_samples_per_second': 24.993, 'train_steps_per_second': 1.6, 'total_flos': 1.9373518305792e+17, 'train_loss': 0.771074716746807, 'epoch': 5.0})

In [46]:
preds = trainer.predict(ds['test'])
print(classification_report(preds.label_ids, preds.predictions.argmax(axis=1)))

              precision    recall  f1-score   support

           0       0.93      0.86      0.90        50
           1       0.12      0.10      0.11        50
           2       0.18      0.30      0.23        50
           3       0.70      0.38      0.49        50
           4       0.89      0.94      0.91        50

    accuracy                           0.52       250
   macro avg       0.57      0.52      0.53       250
weighted avg       0.57      0.52      0.53       250



## Independent Synthetic Data

In [21]:
processed_dataset = syn.map(preprocess_function, batched=True)

model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224', num_labels=5, ignore_mismatched_sizes=True)

# Create the Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=processed_dataset['train'],
    eval_dataset=ds['test'],
    compute_metrics=compute_metrics,
)

# Train the model
trainer.train()

Map:   0%|          | 0/1600 [00:00<?, ? examples/s]

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([5]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([5, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.485500,1.337899,0.560000,0.516359
2,0.166200,1.118210,0.748000,0.735270
3,0.050300,0.983992,0.812000,0.803625
4,0.032900,0.929392,0.832000,0.827122
5,0.038500,0.915038,0.832000,0.827724


TrainOutput(global_step=500, training_loss=0.2779338041841984, metrics={'train_runtime': 272.9266, 'train_samples_per_second': 29.312, 'train_steps_per_second': 1.832, 'total_flos': 6.19952585785344e+17, 'train_loss': 0.2779338041841984, 'epoch': 5.0})

In [33]:
preds = trainer.predict(ds['test'])
print(classification_report(preds.label_ids, preds.predictions.argmax(axis=1)))

              precision    recall  f1-score   support

           0       0.64      0.98      0.77        50
           1       0.92      0.96      0.94        50
           2       0.97      0.56      0.71        50
           3       1.00      0.68      0.81        50
           4       0.84      0.98      0.91        50

    accuracy                           0.83       250
   macro avg       0.87      0.83      0.83       250
weighted avg       0.87      0.83      0.83       250



## 5-shot synthetic data

In [53]:
train_ds = create_extended_dataset(fsl['train'], 3)
processed_dataset = train_ds.map(preprocess_function, batched=True)

model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224', num_labels=5, ignore_mismatched_sizes=True)

# Create the Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=processed_dataset,
    eval_dataset=ds['test'],
    compute_metrics=compute_metrics,
)

# Train the model
trainer.train()

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([5]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([5, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.560400,0.929855,0.868000,0.866584
2,0.185700,0.582899,0.936000,0.935002
3,0.102200,0.415749,0.968000,0.967910
4,0.193300,0.350093,0.976000,0.975971
5,0.141300,0.333270,0.976000,0.975971


TrainOutput(global_step=940, training_loss=0.3243149625058187, metrics={'train_runtime': 491.6144, 'train_samples_per_second': 30.512, 'train_steps_per_second': 1.912, 'total_flos': 1.16241109834752e+18, 'train_loss': 0.3243149625058187, 'epoch': 5.0})

In [54]:
preds = trainer.predict(ds['test'])
print(classification_report(preds.label_ids, preds.predictions.argmax(axis=1)))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        50
           1       0.96      1.00      0.98        50
           2       0.93      1.00      0.96        50
           3       1.00      0.92      0.96        50
           4       1.00      0.96      0.98        50

    accuracy                           0.98       250
   macro avg       0.98      0.98      0.98       250
weighted avg       0.98      0.98      0.98       250

